In [11]:
import gurobipy as gp
from gurobipy import GRB


In [12]:
gates = ['1','2','3','4','5','6','7']
gates47 = ['4','5','6','7']
rows = ['1','2','3','4']
gatesRows, valueA, valueB = gp.multidict({
    ('1','1'): [0,0],
    ('1','2'): [0,1],
    ('1','3'): [1,0],
    ('1','4'): [1,1],
    ('2','1'): [0,0],
    ('2','2'): [0,1],
    ('2','3'): [1,0],
    ('2','4'): [1,1],
    ('3','1'): [0,0],
    ('3','2'): [0,1],
    ('3','3'): [1,0],
    ('3','4'): [1,1],
    ('4','1'): [0,0],
    ('4','2'): [0,1],
    ('4','3'): [1,0],
    ('4','4'): [1,1],
    ('5','1'): [0,0],
    ('5','2'): [0,1],
    ('5','3'): [1,0],
    ('5','4'): [1,1],
    ('6','1'): [0,0],
    ('6','2'): [0,1],
    ('6','3'): [1,0],
    ('6','4'): [1,1],
    ('7','1'): [0,0],
    ('7','2'): [0,1],
    ('7','3'): [1,0],
    ('7','4'): [1,1]
})


In [13]:
model = gp.Model('logicalDesign')

NOR = model.addVars(gates, vtype=GRB.BINARY, name="NORgate" )
NOR['1'].lb = 1
inputA = model.addVars(gates, vtype=GRB.BINARY, name="inputA")
inputB = model.addVars(gates, vtype=GRB.BINARY, name="inputB")
output = model.addVars(gatesRows, vtype=GRB.BINARY, name="output")
output['1','1'].ub = 0
output['1','2'].lb = 1
output['1','3'].lb = 1
output['1','4'].ub = 0

In [14]:
externalInputsA = model.addConstrs( ( NOR[i] >= inputA[i]  for i in gates), name='externalInputsA')
externalInputsB = model.addConstrs( ( NOR[i] >= inputB[i]  for i in gates), name='externalInputsB')

In [15]:
NORgate1 = model.addConstr(NOR['2'] + NOR['3'] + inputA['1'] + inputB['1'] <= 2, name='NORgate1')
NORgate2 = model.addConstr(NOR['4'] + NOR['5'] + inputA['2'] + inputB['2'] <= 2, name='NORgate2')
NORgate3 = model.addConstr(NOR['6'] + NOR['7'] + inputA['3'] + inputB['3'] <= 2, name='NORgate3')

In [16]:
outputSignals1_1 = model.addConstrs( (output['2',r] + output['1',r] <= 1 for r in rows), name='outputSignals1_1' )

outputSignals1_2 = model.addConstrs( (output['3',r] + output['1',r] <= 1 for r in rows), name='outputSignals1_2' )

outputSignals2_1 = model.addConstrs( (output['4',r] + output['2',r] <= 1 for r in rows), name='outputSignals2_1' )

outputSignals2_2 = model.addConstrs( (output['5',r] + output['2',r] <= 1 for r in rows), name='outputSignals2_2' )

outputSignals3_1 = model.addConstrs( (output['6',r] + output['3',r] <= 1 for r in rows), name='outputSignals3_1' )

outputSignals3_2 = model.addConstrs( (output['7',r] + output['3',r] <= 1 for r in rows), name='outputSignals3_2' )


outputSignals4 = model.addConstrs( (valueA[i,r]*inputA[i] + output[i,r] <= 1 for i,r in gatesRows), name='outputSignals4')

outputSignals5 = model.addConstrs( (valueB[i,r]*inputB[i] + output[i,r] <= 1 for i,r in gatesRows), name='outputSignals5')

outputSignals6 = model.addConstrs( (valueA[i,r]*inputA[i] + valueB[i,r]*inputB[i] + output[i,r] - NOR[i] >= 0 
                                    for i,r in gatesRows if i in gates47), name='outputSignals6')

outputSignals7 = model.addConstrs( (valueA['1',r]*inputA['1'] + valueB['1',r]*inputB['1'] 
                                    + output['2',r] + output['3',r] + output['1',r] - NOR['1'] >= 0
                                    for i,r in gatesRows), name='outputSignals7')

outputSignals8 = model.addConstrs( (valueA['2',r]*inputA['2'] + valueB['2',r]*inputB['2'] 
                                    + output['4',r] + output['5',r] + output['2',r] - NOR['2'] >= 0
                                    for i,r in gatesRows), name='outputSignals8')

outputSignals9 = model.addConstrs( (valueA['3',r]*inputA['3'] + valueB['3',r]*inputB['3'] 
                                    + output['6',r] + output['7',r] + output['3',r] - NOR['3'] >= 0
                                    for i,r in gatesRows), name='outputSignals9')

In [17]:
gateOutput = model.addConstrs( (NOR[i] - output[i,r] >= 0 for i,r in gatesRows) , name='gateOutput')

In [18]:
model.setObjective(NOR.sum(), GRB.MINIMIZE)

In [19]:
model.write('logicalDesign.lp')
model.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 225 rows, 49 columns and 696 nonzeros
Model fingerprint: 0x9adba516
Variable types: 0 continuous, 49 integer (49 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 225 rows and 49 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 1: 5 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.000000000000e+00, best bound 5.000000000000e+00, gap 0.0000%


In [20]:
for i in gates:
    if (NOR[i].x > 0.5):
        if (inputA[i].x + inputB[i].x > 0.5):
            print(f"NOR gate {i} is active, with external inputs A and B values of {inputA[i].x} and  {inputB[i].x}.")
        else:
            print(f"NOR gate {i} is active.")

NOR gate 1 is active.
NOR gate 2 is active, with external inputs A and B values of 1.0 and  1.0.
NOR gate 3 is active.
NOR gate 6 is active, with external inputs A and B values of 0.0 and  1.0.
NOR gate 7 is active, with external inputs A and B values of 1.0 and  0.0.
